In [1]:
import numpy as np
import cv2 as cv

In [85]:
img_path = "/home/bot/dev/projects_data/ale/cap/cropped_img.png";
img = cv.imread(img_path, 0)

In [10]:

res1 = np.uint8(np.clip((cv.add(1*img, 30)), 0, 255))
res2 = np.uint8(np.clip((cv.add(1.5*img, 60)), 0, 255))
tmp = np.hstack((img, res1, res2))

cv.imshow('images', tmp)
cv.waitKey(0)
cv.destroyAllWindows()

In [86]:
res = img.copy()

alpha = 0.3
beta = 80

def updateParams(x):
    global alpha, beta, img, res
    alpha = cv.getTrackbarPos('Alpha', 'image') * 0.01
    beta = cv.getTrackbarPos('Beta', 'image')
    res = np.uint8(np.clip((alpha * img + beta), 0, 250))
    
cv.namedWindow('image')
cv.createTrackbar('Alpha', 'image', 0, 300, updateParams)
cv.createTrackbar('Beta', 'image', 0, 255, updateParams)
cv.setTrackbarPos('Alpha', 'image', 100)
cv.setTrackbarPos('Beta', 'image', 0)

while True:
    cv.imshow('image', res)
    if cv.waitKey(1) == ord('q'):
        break
cv.destroyAllWindows()

print(alpha)
print(beta)


2.39
3


In [87]:
tmp = np.hstack((img, res))
cv.imshow('images', tmp)
cv.waitKey(0)
cv.destroyAllWindows()

In [18]:
print(res.shape)

(480, 640)


In [89]:
blur = cv.GaussianBlur(res,(5,5),0)
tmp = np.hstack((img, res, blur))
cv.imshow('images', tmp)
cv.waitKey(0)
cv.destroyAllWindows()

In [90]:
low = 50
high = 150

# edges = cv.Canny(res, low, high)

def updateCannyParams(x):
    global low, high, res, edges
    low = cv.getTrackbarPos('low', 'Canny')
    high = cv.getTrackbarPos('high', 'Canny')
    edges = cv.Canny(blur, low, high)
    
cv.namedWindow('Canny')
cv.createTrackbar('low', 'Canny', 0, 255, updateCannyParams)
cv.createTrackbar('high', 'Canny', 0, 255, updateCannyParams)
cv.setTrackbarPos('low', 'Canny', 50)
cv.setTrackbarPos('high', 'Canny', 100)

while True:
    cv.imshow('Canny', edges)
    if cv.waitKey(1) == ord('q'):
        break
cv.destroyAllWindows()

print(low)
print(high)


32
255


In [93]:

contours, hierarchy = cv.findContours(edges, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
print(len(contours))

# img_with_contours = np.zeros(img.shape)
img_with_contours = np.zeros((img.shape[0], img.shape[1], 3), np.uint8)
cv.drawContours(img_with_contours, contours, -1, (255,255,255), 3)

# tmp = np.hstack((edges, binary_img, img_with_contours))
# cv.imshow('img_with_contours', img_with_contours)
# cv.waitKey(0)
# cv.destroyAllWindows()

img_with_contours = cv.cvtColor(img_with_contours, cv.COLOR_RGB2GRAY)

print(img_with_contours.shape)

8
(480, 640)


In [94]:
index = 0

font=cv.FONT_HERSHEY_SIMPLEX

def angle_cos(p0, p1, p2):
    d1, d2 = (p0-p1).astype('float'), (p2-p1).astype('float')
    return abs( np.dot(d1, d2) / np.sqrt( np.dot(d1, d1)*np.dot(d2, d2) ) )


squares = []
for cnt in contours:
    cnt_len = cv.arcLength(cnt, True) #计算轮廓周长
    cnt = cv.approxPolyDP(cnt, 0.02*cnt_len, True) #多边形逼近
    print('-------------------')
    print(len(cnt))
    print(cv.isContourConvex(cnt))
    
    # 条件判断逼近边的数量是否为4，轮廓面积是否大于1000，检测轮廓是否为凸的
#     if len(cnt) == 4 and cv.contourArea(cnt) > 100 and cv.isContourConvex(cnt):
    if len(cnt) == 4 and cv.isContourConvex(cnt):

        M = cv.moments(cnt) #计算轮廓的矩
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])#轮廓重心

        cnt = cnt.reshape(-1, 2)
        max_cos = np.max([angle_cos( cnt[i], cnt[(i+1) % 4], cnt[(i+2) % 4] ) for i in range(4)])
        
        print('--')
        print(max_cos)
        
        # 只检测矩形（cos90° = 0）
        if max_cos < 0.1:
        # 检测四边形（不限定角度范围）
#         if True:
            index = index + 1
#             cv.putText(img_with_contours,("#%d"%index),(cx,cy),font,0.7,(255,0,255),2)
            squares.append(cnt)
            
    cur_contour = [cnt]
    imggggg = np.zeros(img.shape)
    cv.drawContours(imggggg, cur_contour, -1, (255,255,255), 1)
    tmp = np.hstack((edges, imggggg))
    cv.imshow('imggggg', tmp)
    cv.waitKey(0)


print()
print(index)

tmp = np.hstack((edges, binary_img, img_with_contours))
cv.imshow('imgs', tmp)
cv.waitKey(0)
cv.destroyAllWindows()

-------------------
4
True
--
0.00651960743827118
-------------------
4
True
--
0.00727552985765212
-------------------
4
False
-------------------
4
True
--
0.05126330732211238
-------------------
4
True
--
0.029315925477634064
-------------------
2
False
-------------------
4
True
--
0.0706689478089619
-------------------
4
True
--
0.04699621263752874

6


In [60]:
help(cv.connectedComponentsWithStats)

Help on built-in function connectedComponentsWithStats:

connectedComponentsWithStats(...)
    connectedComponentsWithStats(image[, labels[, stats[, centroids[, connectivity[, ltype]]]]]) -> retval, labels, stats, centroids
    .   @overload
    .   @param image the 8-bit single-channel image to be labeled
    .   @param labels destination labeled image
    .   @param stats statistics output for each label, including the background label, see below for
    .   available statistics. Statistics are accessed via stats(label, COLUMN) where COLUMN is one of
    .   #ConnectedComponentsTypes. The data type is CV_32S.
    .   @param centroids centroid output for each label, including the background label. Centroids are
    .   accessed via centroids(label, 0) for x and centroids(label, 1) for y. The data type CV_64F.
    .   @param connectivity 8 or 4 for 8-way or 4-way connectivity respectively
    .   @param ltype output image label type. Currently CV_32S and CV_16U are supported.



In [95]:
invert_contours = 255 - img_with_contours
num_objects, labels, stats, centroids = cv.connectedComponentsWithStats(invert_contours, connectivity=8)
print('num:',num_objects)
print('stats:',stats)
print('centroids:',centroids)
print('labels:',labels)

num: 23
stats: [[   245    111    186    227   8006]
 [     0      0    640    480 275235]
 [   379    117      1      1      1]
 [   384    116     25     89    221]
 [   368    119      2      1      2]
 [   373    118      2      1      2]
 [   362    120      2      1      2]
 [   257    122    145    108  11385]
 [   250    140     17     91    168]
 [   374    209     52    101    344]
 [   364    221      3      1      3]
 [   369    220      3      1      3]
 [   274    220    144    108  11461]
 [   352    223      3      1      3]
 [   358    222      3      1      3]
 [   340    225      3      1      3]
 [   346    224      3      1      3]
 [   269    226     53     13    184]
 [   325    227      8      2      8]
 [   335    226      3      1      3]
 [   268    245     19     88    158]
 [   416    310      1      1      1]
 [   399    313      1      1      1]]
centroids: [[337.61766175 223.87596802]
 [317.39614511 241.28049485]
 [379.         117.        ]
 [397.941176

In [96]:
output = np.zeros((img.shape[0], img.shape[1], 3), np.uint8)
for i in range(0, num_objects):
    mask = labels == i
    output[:,:,0][mask] = np.random.randint(0,255)
    output[:,:,1][mask] = np.random.randint(0,255)
    output[:,:,2][mask] = np.random.randint(0,255)

cv.imshow('img', img)
cv.imshow('invert_contours', invert_contours)
cv.imshow('output', output)
cv.waitKey(0)
cv.destroyAllWindows()